# MongoDB Atlas

本Jupyter Notebook介绍了如何在LangChain中使用`langchain-mongodb`包进行MongoDB Atlas向量搜索。

[MongoDB Atlas](https://www.mongodb.com/docs/atlas/)是一个完全托管的云数据库，可在AWS、Azure和GCP上使用。它支持对MongoDB文档数据进行原生向量搜索、全文搜索（BM25）和混合搜索。

[MongoDB Atlas向量搜索](https://www.mongodb.com/products/platform/atlas-vector-search)允许您将嵌入存储在MongoDB文档中，创建向量搜索索引，并使用近似最近邻算法（`分层可导航小世界`）执行KNN搜索。它使用[$vectorSearch MQL阶段](https://www.mongodb.com/docs/atlas/atlas-vector-search/vector-search-overview/)

## 设置

运行MongoDB 6.0.11、7.0.2或更高版本（包括RCs）的Atlas集群。

要使用MongoDB Atlas，您必须首先部署一个集群。我们提供一个免费层级的集群，您可以在选择的云上使用。要开始使用，请访问Atlas的[快速入门](https://www.mongodb.com/docs/atlas/getting-started/)。

您需要安装`langchain-mongodb`和`pymongo`才能使用此集成。

In [ ]:
pip install -qU langchain-mongodb pymongo

### 凭据

对于本Notebook，您需要找到您的MongoDB集群URI。

有关查找集群URI的信息，请阅读[此指南](https://www.mongodb.com/docs/manual/reference/connection-string/)

In [33]:
import getpass

MONGODB_ATLAS_CLUSTER_URI = getpass.getpass("MongoDB Atlas 集群 URI:")

如果您想获得一流的模型调用自动化追踪，您还可以通过取消注释以下内容来设置您的[LangSmith](https://docs.smith.langchain.com/) API密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("请输入您的LangSmith API密钥：")
# os.environ["LANGSMITH_TRACING"] = "true"

## 初始化
```
import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs/>
```

In [54]:
# | output: false
# | echo: false
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [56]:
from langchain_mongodb import MongoDBAtlasVectorSearch
from pymongo import MongoClient

# 初始化MongoDB Python客户端
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

DB_NAME = "langchain_test_db"
COLLECTION_NAME = "langchain_test_vectorstores"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "langchain-test-index-vectorstores"

MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

vector_store = MongoDBAtlasVectorSearch(
    collection=MONGODB_COLLECTION,
    embedding=embeddings,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    relevance_score_fn="cosine",
)

# 在集合上创建向量搜索索引
# 由于我们使用的是默认的OpenAI嵌入模型（ada-v2），因此需要将维度指定为1536
vector_store.create_vector_search_index(dimensions=1536)

[可选] 除了上述`vector_store.create_vector_search_index`命令之外，您还可以使用Atlas UI并采用以下索引定义来创建向量搜索索引：
```json
{
  "fields":[
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    }
  ]
}
```

## 管理向量存储

创建向量存储后，我们可以通过添加和删除不同的项目来与其交互。

### 向向量存储添加项目

我们可以使用`add_documents`函数向向量存储添加项目。

In [57]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="我今天早上吃了巧克力片煎饼和炒鸡蛋。",
    metadata={"source": "推文"},
)

document_2 = Document(
    page_content="明天的天气预报是多云转阴，最高气温62华氏度。",
    metadata={"source": "新闻"},
)

document_3 = Document(
    page_content="正在用LangChain构建一个令人兴奋的新项目——快来看看吧！",
    metadata={"source": "推文"},
)

document_4 = Document(
    page_content="劫匪闯入市银行，偷走了100万美元现金。",
    metadata={"source": "新闻"},
)

document_5 = Document(
    page_content="哇！那部电影太棒了。我等不及要再看一遍。",
    metadata={"source": "推文"},
)

document_6 = Document(
    page_content="新款iPhone值得这个价格吗？阅读这篇评论来了解一下。",
    metadata={"source": "网站"},
)

document_7 = Document(
    page_content="目前世界上排名前10的足球运动员。",
    metadata={"source": "网站"},
)

document_8 = Document(
    page_content="LangGraph是构建有状态、代理式应用程序的最佳框架！",
    metadata={"source": "推文"},
)

document_9 = Document(
    page_content="由于对经济衰退的担忧，今天股市下跌了500点。",
    metadata={"source": "新闻"},
)

document_10 = Document(
    page_content="我有一种不好的预感，我会被删除 :(",
    metadata={"source": "推文"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['03ad81e8-32a0-46f0-b7d8-f5b977a6b52a',
 '8396a68d-f4a3-4176-a581-a1a8c303eea4',
 'e7d95150-67f6-499f-b611-84367c50fa60',
 '8c31b84e-2636-48b6-8b99-9fccb47f7051',
 'aa02e8a2-a811-446a-9785-8cea0faba7a9',
 '19bd72ff-9766-4c3b-b1fd-195c732c562b',
 '642d6f2f-3e34-4efa-a1ed-c4ba4ef0da8d',
 '7614bb54-4eb5-4b3b-990c-00e35cb31f99',
 '69e18c67-bf1b-43e5-8a6e-64fb3f240e52',
 '30d599a7-4a1a-47a9-bbf8-6ed393e2e33c']

### 从向量存储中删除项目


In [58]:
vector_store.delete(ids=[uuids[-1]])

True

## 查询向量存储

一旦您的向量存储创建并添加了相关文档，您很可能希望在运行您的链或代理时对其进行查询。 

### 直接查询

#### 相似性搜索

执行简单的相似性搜索可以按如下方式进行：

In [62]:
results = vector_store.similarity_search("LangChain提供了抽象层，使LLM的使用变得容易", k=2)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* 正在用LangChain构建一个令人兴奋的新项目——快来看看吧！ [{'_id': 'e7d95150-67f6-499f-b611-84367c50fa60', 'source': '推文'}]
* LangGraph是构建有状态、代理式应用程序的最佳框架！ [{'_id': '7614bb54-4eb5-4b3b-990c-00e35cb31f99', 'source': '推文'}]


#### 带分数的相似性搜索

您还可以带分数进行搜索：

In [63]:
results = vector_store.similarity_search_with_score("明天会很热吗？", k=1)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.784560] 明天的天气预报是多云转阴，最高气温62华氏度。 [{'_id': '8396a68d-f4a3-4176-a581-a1a8c303eea4', 'source': '新闻'}]


### 带有预过滤的相似性搜索

Atlas向量搜索支持使用MQL运算符进行预过滤。下面是一个在上述相同加载数据上进行索引和查询的示例，允许您对“page”字段进行元数据过滤。您可以更新现有索引以包含定义的过滤器，并使用向量搜索进行预过滤。



要启用预过滤，您需要更新索引定义以包含一个过滤器字段。在此示例中，我们将使用`source`字段作为过滤器字段。

这可以通过编程方式使用`MongoDBAtlasVectorSearch.create_vector_search_index`方法完成。

```python
vectorstore.create_vector_search_index(
  dimensions=1536,
  filters=[{"type":"filter", "path":"source"}],
  update=True
)
```

或者，您也可以使用Atlas UI并通过以下索引定义来更新索引：
```json
{
  "fields":[
    {
      "type": "vector",
      "path": "embedding",
      "numDimensions": 1536,
      "similarity": "cosine"
    },
    {
      "type": "filter",
      "path": "source"
    }
  ]
}
```

然后您可以按如下方式运行带过滤器的查询：

```python
results = vector_store.similarity_search(query="foo", k=1, pre_filter={"source": {"$eq": "https://example.com"}})
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")
```

#### 其他搜索方法

本Notebook中未涵盖多种其他搜索方法，例如MMR搜索或按向量搜索。有关`MongoDBAtlasVectorStore`可用搜索功能的完整列表，请查看[API参考](https://python.langchain.com/api_reference/mongodb/vectorstores/langchain_mongodb.vectorstores.MongoDBAtlasVectorSearch.html)。

### 通过转换为检索器进行查询

您还可以将向量存储转换为检索器，以便在您的链中更方便地使用。 

以下是如何将向量存储转换为检索器，然后使用简单的查询和过滤器调用该检索器。

In [65]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.2},
)
retriever.invoke("从银行偷窃是犯罪行为")

[Document(metadata={'_id': '8c31b84e-2636-48b6-8b99-9fccb47f7051', 'source': '新闻'}, page_content='劫匪闯入市银行，偷走了100万美元现金。')]

## 用于检索增强生成（RAG）的用法

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程](/docs/tutorials/)
- [操作指南：使用RAG进行问答](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [检索概念文档](https://python.langchain.com/docs/concepts/retrieval)

# 其他注意事项
* 更多文档可在[MongoDB的LangChain文档](https://www.mongodb.com/docs/atlas/atlas-vector-search/ai-integrations/langchain/)网站上找到
* 此功能已普遍可用，可用于生产部署。
* Langchain `0.0.305`版本（[发行说明](https://github.com/langchain-ai/langchain/releases/tag/v0.0.305)）引入了对$vectorSearch MQL阶段的支持，该阶段适用于MongoDB Atlas `6.0.11`和`7.0.2`。使用早期MongoDB Atlas版本的用户需要将其LangChain版本固定到<=`0.0.304`。
> 

## API参考

有关所有`MongoDBAtlasVectorSearch`功能和配置的详细文档，请前往API参考：https://python.langchain.com/api_reference/mongodb/index.html